# 🎯 Streamlined UFC Betting Analysis

**Professional UFC betting analysis with dynamic bankroll management**

- ✅ **Cell 1**: Setup and model loading
- 📊 **Cell 2**: Live odds fetching with CSV storage  
- 🧠 **Cell 3**: Predictions with favorite identification
- 💰 **Cell 4**: Dynamic bankroll management (research-backed Kelly)
- 📝 **Cell 5**: Bet tracking and CSV integration

---

In [1]:
# 🚀 CELL 1: SETUP AND MODEL LOADING
# ===================================
# Professional UFC betting system initialization

import pandas as pd
import numpy as np
import joblib
import json
import warnings
import sys
import os
import uuid
import shutil
from pathlib import Path
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple
warnings.filterwarnings('ignore')

# Add project root to path
sys.path.append('..')

print("🎯 STREAMLINED UFC BETTING SYSTEM")
print("=" * 50)

# Configuration
YOUR_ODDS_API_KEY = "5100c18e74058e57c1d33a747e8c2be1"
INITIAL_BANKROLL = 21.48  # Adjust your bankroll here

def get_latest_trained_models():
    """Auto-detect latest trained models with versioning"""
    model_dir = Path('.')
    training_dirs = [d for d in model_dir.iterdir() if d.is_dir() and d.name.startswith('training_')]
    
    if not training_dirs:
        print("⚠️  No training directories found, using standard model locations...")
        return {
            'winner_model_path': 'ufc_random_forest_model_tuned.joblib',
            'method_model_path': 'ufc_multiclass_model.joblib', 
            'winner_cols_path': 'winner_model_columns.json',
            'method_cols_path': 'method_model_columns.json',
            'fighters_data_path': 'ufc_fighters_engineered_corrected.csv',
            'version': 'standard'
        }
    
    latest_training_dir = sorted(training_dirs, key=lambda x: x.name)[-1]
    version = latest_training_dir.name.replace('training_', '')
    
    return {
        'winner_model_path': latest_training_dir / f'ufc_winner_model_tuned_{version}.joblib',
        'method_model_path': latest_training_dir / f'ufc_method_model_{version}.joblib',
        'winner_cols_path': latest_training_dir / f'winner_model_columns_{version}.json',
        'method_cols_path': latest_training_dir / f'method_model_columns_{version}.json',
        'fighters_data_path': latest_training_dir / f'ufc_fighters_engineered_{version}.csv',
        'version': version
    }

def initialize_betting_system():
    """Initialize complete betting system with error handling"""
    try:
        print("📊 Loading models and data...")
        
        # Get model paths
        paths = get_latest_trained_models()
        
        # Load fighter data
        fighters_data_path = paths['fighters_data_path']
        if not fighters_data_path.exists():
            fighters_data_path = Path('ufc_fighters_engineered_corrected.csv')
        
        fighters_df = pd.read_csv(fighters_data_path)
        print(f"✅ Loaded {len(fighters_df):,} fighter records")
        
        # Load column configurations  
        with open(paths['winner_cols_path'], 'r') as f:
            winner_cols = json.load(f)
        
        with open(paths['method_cols_path'], 'r') as f:
            method_cols = json.load(f)
        
        # Load trained models
        winner_model = joblib.load(paths['winner_model_path'])
        method_model = joblib.load(paths['method_model_path'])
        
        print(f"✅ Models loaded from version: {paths['version']}")
        print(f"   🏆 Winner Model: {winner_model.__class__.__name__} ({len(winner_cols)} features)")
        print(f"   ⚔️  Method Model: {method_model.__class__.__name__} ({len(method_cols)} features)")
        
        # Initialize API client
        print(f"🔌 Initializing Odds API client...")
        from src.odds_api_integration import UFCOddsAPIClient
        
        odds_client = UFCOddsAPIClient(YOUR_ODDS_API_KEY)
        print(f"✅ API client ready (Key: {YOUR_ODDS_API_KEY[:8]}...)")
        
        # Import prediction function
        from src.prediction import predict_fight_symmetrical
        print(f"✅ Prediction function imported")
        
        return {
            'fighters_df': fighters_df,
            'winner_cols': winner_cols,
            'method_cols': method_cols,
            'winner_model': winner_model,
            'method_model': method_model,
            'odds_client': odds_client,
            'predict_function': predict_fight_symmetrical,
            'model_version': paths['version'],
            'bankroll': INITIAL_BANKROLL,
            'api_key': YOUR_ODDS_API_KEY
        }
        
    except Exception as e:
        print(f"❌ System initialization failed: {str(e)}")
        print(f"💡 Check that all required files exist and API key is valid")
        return None

def create_required_directories():
    """Create necessary directories for data storage"""
    directories = [
        Path('odds'),
        Path('betting_backups'),
        Path('analysis_exports')
    ]
    
    for directory in directories:
        directory.mkdir(exist_ok=True)
        print(f"📁 Directory ready: {directory}")

# Initialize system
print("🏗️  Creating required directories...")
create_required_directories()

print("\\n🚀 Initializing betting system...")
betting_system = initialize_betting_system()

if betting_system:
    print(f"\\n✅ SYSTEM READY!")
    print(f"💳 Bankroll: ${betting_system['bankroll']:,}")
    print(f"🎯 Model Version: {betting_system['model_version']}")
    print(f"🔑 API Status: Connected")
    print(f"\\n🎮 Ready for betting analysis!")
else:
    print(f"\\n❌ SYSTEM FAILED TO INITIALIZE")
    print(f"Cannot proceed with betting analysis.")

🎯 STREAMLINED UFC BETTING SYSTEM
🏗️  Creating required directories...
📁 Directory ready: odds
📁 Directory ready: betting_backups
📁 Directory ready: analysis_exports
\n🚀 Initializing betting system...
📊 Loading models and data...
✅ Loaded 4,378 fighter records
✅ Models loaded from version: 2025-07-30_17-06
   🏆 Winner Model: RandomForestClassifier (70 features)
   ⚔️  Method Model: RandomForestClassifier (70 features)
🔌 Initializing Odds API client...
✅ API client ready (Key: 5100c18e...)
✅ Prediction function imported
\n✅ SYSTEM READY!
💳 Bankroll: $21.48
🎯 Model Version: 2025-07-30_17-06
🔑 API Status: Connected
\n🎮 Ready for betting analysis!


In [2]:
# 📊 CELL 2: LIVE ODDS FETCHING WITH CSV STORAGE  
# ================================================
# Fetch live UFC odds and store in organized CSV format

# Configuration - Set your event details here
EVENT_NAME = "UFC_Fight_Night_Albazi_vs_Taira"  # Updated event name
TARGET_FIGHTS = [
    "Amir Albazi vs Tatsuro Taira",
    "Mateusz Rebecki vs Chris Duncan", 
    "Elves Brener vs Esteban Ribovics",
    "Karol Rosa vs Nora Cornolle",
    "Neil Magny vs Elizeu Zaleski dos Santos",
    "Danny Silva vs Kevin Vallejos"
]

def fetch_and_store_odds(event_name: str, target_fights: List[str] = None):
    """
    Fetch live odds and store in organized CSV format
    
    Returns:
        dict: Odds data with metadata
    """
    print("📊 LIVE ODDS FETCHING SYSTEM")
    print("=" * 50)
    
    if not betting_system:
        raise Exception("❌ Betting system not initialized. Run Cell 1 first.")
    
    timestamp = datetime.now()
    
    try:
        # Fetch odds from The Odds API
        print(f"🔌 Fetching odds from The Odds API...")
        print(f"📅 Event: {event_name}")
        print(f"🔑 API Key: {betting_system['api_key'][:8]}...")
        
        # Get UFC odds
        api_data = betting_system['odds_client'].get_ufc_odds(region="au")
        
        # Format for target fights
        formatted_odds = betting_system['odds_client'].format_odds_for_analysis(
            api_data, target_fights
        )
        
        if not formatted_odds:
            raise Exception(
                f"❌ No odds found for target fights\\n"
                f"Available events: {len(api_data)}\\n" 
                f"Target fights: {target_fights}"
            )
        
        print(f"✅ Retrieved odds for {len(formatted_odds)} fights")
        
        # Prepare data for CSV storage
        odds_records = []
        
        for fight_key, fight_data in formatted_odds.items():
            # Create record for fighter A
            odds_records.append({
                'timestamp': timestamp.isoformat(),
                'event_name': event_name,
                'fight_key': fight_key,
                'fighter': fight_data['fighter_a'],
                'opponent': fight_data['fighter_b'], 
                'decimal_odds': fight_data['fighter_a_decimal_odds'],
                'american_odds': decimal_to_american(fight_data['fighter_a_decimal_odds']),
                'implied_probability': 1 / fight_data['fighter_a_decimal_odds'],
                'bookmakers': ', '.join(fight_data.get('bookmakers', ['Unknown'])),
                'position': 'fighter_a'
            })
            
            # Create record for fighter B
            odds_records.append({
                'timestamp': timestamp.isoformat(),
                'event_name': event_name,
                'fight_key': fight_key,
                'fighter': fight_data['fighter_b'],
                'opponent': fight_data['fighter_a'],
                'decimal_odds': fight_data['fighter_b_decimal_odds'],
                'american_odds': decimal_to_american(fight_data['fighter_b_decimal_odds']),
                'implied_probability': 1 / fight_data['fighter_b_decimal_odds'],
                'bookmakers': ', '.join(fight_data.get('bookmakers', ['Unknown'])),
                'position': 'fighter_b'
            })
        
        # Create event-specific odds folder
        event_folder = Path('odds') / event_name
        event_folder.mkdir(exist_ok=True)
        
        # Save to CSV with timestamp
        csv_filename = f"odds_{timestamp.strftime('%Y%m%d_%H%M%S')}.csv"
        csv_path = event_folder / csv_filename
        
        df_odds = pd.DataFrame(odds_records)
        df_odds.to_csv(csv_path, index=False)
        
        print(f"💾 Odds saved to: {csv_path}")
        
        # Display odds summary
        print(f"\\n📋 ODDS SUMMARY")
        print("=" * 30)
        
        for fight_key, fight_data in formatted_odds.items():
            fighter_a = fight_data['fighter_a']
            fighter_b = fight_data['fighter_b']
            odds_a = fight_data['fighter_a_decimal_odds']
            odds_b = fight_data['fighter_b_decimal_odds']
            
            # Identify favorite
            if odds_a < odds_b:
                favorite = f"⭐ {fighter_a}"
                underdog = fighter_b
            else:
                favorite = f"⭐ {fighter_b}"
                underdog = fighter_a
            
            print(f"\\n🥊 {fight_key}")
            print(f"   {favorite} ({min(odds_a, odds_b):.2f}) vs {underdog} ({max(odds_a, odds_b):.2f})")
            print(f"   Market edge: {abs(odds_a - odds_b):.2f}")
        
        return {
            'status': 'success',
            'odds_data': formatted_odds,
            'csv_path': str(csv_path),
            'event_name': event_name,
            'total_fights': len(formatted_odds),
            'fetch_timestamp': timestamp.isoformat()
        }
        
    except Exception as e:
        print(f"❌ Odds fetching failed: {str(e)}")
        return {
            'status': 'failed',
            'error': str(e),
            'event_name': event_name
        }

def decimal_to_american(decimal_odds: float) -> int:
    """Convert decimal odds to American format"""
    if decimal_odds >= 2.0:
        return int((decimal_odds - 1) * 100)
    else:
        return int(-100 / (decimal_odds - 1))

def display_api_usage():
    """Display current API usage"""
    try:
        # Make a quick API call to check usage
        response = betting_system['odds_client'].get_ufc_odds(region="au")
        print(f"\\n📊 API USAGE STATUS")
        print(f"Connection: ✅ Active")
        print(f"Available Events: {len(response) if response else 0}")
    except Exception as e:
        print(f"\\n⚠️  API Usage Check Failed: {str(e)}")

# Execute odds fetching
print(f"🎯 Fetching odds for {EVENT_NAME}...")
print(f"Target fights: {len(TARGET_FIGHTS)}")

# Check API status first
display_api_usage()

# Fetch and store odds
odds_result = fetch_and_store_odds(EVENT_NAME, TARGET_FIGHTS)

if odds_result['status'] == 'success':
    print(f"\\n✅ ODDS FETCHING COMPLETE!")
    print(f"📊 Fights processed: {odds_result['total_fights']}")
    print(f"💾 Data stored: {odds_result['csv_path']}")
    print(f"\\n🎮 Ready for predictions (Cell 3)!")
else:
    print(f"\\n❌ ODDS FETCHING FAILED")
    print(f"Error: {odds_result['error']}")
    print(f"\\n💡 Check your API key and internet connection")

🎯 Fetching odds for UFC_Fight_Night_Albazi_vs_Taira...
Target fights: 6
🔄 Fetching UFC odds from The Odds API...
   Region: au
   URL: https://api.the-odds-api.com/v4/sports/mma_mixed_martial_arts/odds
📊 API Usage - Remaining: 499, Used: 1
✅ Found 104 UFC events
\n📊 API USAGE STATUS
Connection: ✅ Active
Available Events: 104
📊 LIVE ODDS FETCHING SYSTEM
🔌 Fetching odds from The Odds API...
📅 Event: UFC_Fight_Night_Albazi_vs_Taira
🔑 API Key: 5100c18e...
🔄 Fetching UFC odds from The Odds API...
   Region: au
   URL: https://api.the-odds-api.com/v4/sports/mma_mixed_martial_arts/odds
📊 API Usage - Remaining: 498, Used: 2
✅ Found 104 UFC events
📋 Karol Rosa vs Nora Cornolle
   Karol Rosa: 1.57
   Nora Cornolle: 2.63
   Bookmakers: Neds, PointsBet (AU), TABtouch, SportsBet, Unibet, PlayUp, Ladbrokes, Betr, Betfair
📋 Mateusz Rebecki vs Chris Duncan
   Mateusz Rebecki: 1.5
   Chris Duncan: 2.88
   Bookmakers: BoomBet, Neds, PointsBet (AU), TABtouch, SportsBet, Unibet, PlayUp, Ladbrokes, Betr, B

In [3]:
# 🧠 CELL 3: PREDICTIONS WITH FAVORITE IDENTIFICATION
# ==================================================
# Generate model predictions and analyze vs market odds

def generate_fight_predictions():
    """
    Generate predictions for all fights with market analysis
    
    Returns:
        dict: Complete prediction analysis
    """
    print("🧠 PREDICTION ENGINE WITH MARKET ANALYSIS")
    print("=" * 50)
    
    if not betting_system:
        raise Exception("❌ Betting system not initialized. Run Cell 1 first.")
    
    if odds_result['status'] != 'success':
        raise Exception("❌ No odds data available. Run Cell 2 first.")
    
    predictions_analysis = {
        'event_name': odds_result['event_name'],
        'timestamp': datetime.now().isoformat(),
        'fight_predictions': [],
        'summary': {
            'total_fights': 0,
            'successful_predictions': 0,
            'failed_predictions': 0,
            'upset_opportunities': 0,
            'high_confidence_picks': 0,
            'method_breakdown': {}
        }
    }
    
    print(f"📅 Event: {odds_result['event_name']}")
    print(f"🎯 Generating predictions for {len(odds_result['odds_data'])} fights...")
    
    for fight_key, odds_data in odds_result['odds_data'].items():
        fighter_a = odds_data['fighter_a']
        fighter_b = odds_data['fighter_b']
        
        print(f"\\n🥊 {fight_key}")
        print(f"   Analyzing: {fighter_a} vs {fighter_b}")
        
        try:
            # Generate model prediction using existing function
            prediction = betting_system['predict_function'](
                fighter_a, fighter_b,
                betting_system['fighters_df'],
                betting_system['winner_cols'],
                betting_system['method_cols'],
                betting_system['winner_model'],
                betting_system['method_model']
            )
            
            if 'error' in prediction:
                print(f"   ❌ Prediction failed: {prediction['error']}")
                predictions_analysis['summary']['failed_predictions'] += 1
                continue
            
            # Extract model probabilities
            model_prob_a = float(prediction['win_probabilities'][fighter_a].replace('%', '')) / 100
            model_prob_b = float(prediction['win_probabilities'][fighter_b].replace('%', '')) / 100
            
            # Get market odds and probabilities
            market_odds_a = odds_data['fighter_a_decimal_odds']
            market_odds_b = odds_data['fighter_b_decimal_odds']
            market_prob_a = 1 / market_odds_a
            market_prob_b = 1 / market_odds_b
            
            # Determine favorites
            market_favorite = fighter_a if market_odds_a < market_odds_b else fighter_b
            model_favorite = fighter_a if model_prob_a > model_prob_b else fighter_b
            
            model_favorite_prob = max(model_prob_a, model_prob_b)
            market_favorite_prob = max(market_prob_a, market_prob_b)
            
            # Calculate expected values
            ev_a = (model_prob_a * market_odds_a) - 1
            ev_b = (model_prob_b * market_odds_b) - 1
            
            # Determine opportunity types
            is_upset_opportunity = (
                market_favorite != model_favorite and 
                model_favorite_prob > 0.55 and
                max(ev_a, ev_b) > 0.08
            )
            
            is_high_confidence = (
                model_favorite_prob > 0.7 and
                market_favorite == model_favorite and
                max(ev_a, ev_b) > 0.05
            )
            
            # Confidence score calculation
            prob_diff = abs(model_prob_a - model_prob_b)
            market_model_alignment = 1.0 if market_favorite == model_favorite else 0.5
            confidence_score = (prob_diff + market_model_alignment) / 2
            
            # Method prediction
            predicted_method = prediction['predicted_method']
            if predicted_method not in predictions_analysis['summary']['method_breakdown']:
                predictions_analysis['summary']['method_breakdown'][predicted_method] = 0
            predictions_analysis['summary']['method_breakdown'][predicted_method] += 1
            
            # Create comprehensive prediction record
            fight_prediction = {
                'fight_key': fight_key,
                'fighter_a': fighter_a,
                'fighter_b': fighter_b,
                
                # Model predictions
                'model_prediction_a': model_prob_a,
                'model_prediction_b': model_prob_b,
                'model_favorite': model_favorite,
                'model_favorite_prob': model_favorite_prob,
                'predicted_method': predicted_method,
                
                # Market data
                'market_odds_a': market_odds_a,
                'market_odds_b': market_odds_b,
                'market_prob_a': market_prob_a,
                'market_prob_b': market_prob_b,
                'market_favorite': market_favorite,
                'market_favorite_prob': market_favorite_prob,
                
                # Analysis
                'expected_value_a': ev_a,
                'expected_value_b': ev_b,
                'confidence_score': confidence_score,
                'is_upset_opportunity': is_upset_opportunity,
                'is_high_confidence': is_high_confidence,
                
                # Raw prediction data
                'raw_prediction': prediction
            }
            
            predictions_analysis['fight_predictions'].append(fight_prediction)
            predictions_analysis['summary']['successful_predictions'] += 1
            
            if is_upset_opportunity:
                predictions_analysis['summary']['upset_opportunities'] += 1
            if is_high_confidence:
                predictions_analysis['summary']['high_confidence_picks'] += 1
            
            # Display results
            print(f"   🏆 Model Favorite: {model_favorite} ({model_favorite_prob:.1%})")
            print(f"   📊 Market Favorite: {market_favorite} ({market_favorite_prob:.1%})")
            print(f"   ⚔️  Method: {predicted_method}")
            print(f"   🎯 Confidence: {confidence_score:.1%}")
            
            if is_upset_opportunity:
                print(f"   🚨 UPSET OPPORTUNITY!")
            if is_high_confidence:
                print(f"   ⭐ HIGH CONFIDENCE PICK!")
                
            # Show expected values
            best_ev = max(ev_a, ev_b)
            best_fighter = fighter_a if ev_a > ev_b else fighter_b
            if best_ev > 0.05:
                print(f"   💎 Best EV: {best_fighter} ({best_ev:+.1%})")
            
        except Exception as e:
            print(f"   ❌ Error generating prediction: {str(e)}")
            predictions_analysis['summary']['failed_predictions'] += 1
    
    # Update summary
    predictions_analysis['summary']['total_fights'] = len(odds_result['odds_data'])
    
    # Display final summary
    print(f"\\n📈 PREDICTION ANALYSIS SUMMARY")
    print("=" * 40)
    summary = predictions_analysis['summary']
    print(f"Total Fights: {summary['total_fights']}")
    print(f"Successful Predictions: {summary['successful_predictions']}")
    print(f"Failed Predictions: {summary['failed_predictions']}")
    print(f"Upset Opportunities: {summary['upset_opportunities']}")
    print(f"High Confidence Picks: {summary['high_confidence_picks']}")
    
    if summary['method_breakdown']:
        print(f"\\n🥊 Method Predictions:")
        for method, count in summary['method_breakdown'].items():
            percentage = (count / summary['successful_predictions']) * 100
            print(f"   {method}: {count} fights ({percentage:.1f}%)")
    
    return predictions_analysis

def display_fight_comparison_table(predictions_data):
    """Display formatted comparison table with corrected formatting"""
    print(f"\\n📊 DETAILED FIGHT ANALYSIS")
    print("=" * 80)
    
    for fight in predictions_data['fight_predictions']:
        print(f"\\n🥊 {fight['fight_key']}")
        print(f"{'Fighter':<20} {'Model':<8} {'Market':<8} {'EV':<8} {'Status'}")
        print("-" * 60)
        
        # Fighter A - CORRECTED F-STRING FORMATTING
        status_a = ""
        if fight['model_favorite'] == fight['fighter_a']:
            status_a += "⭐"
        if fight['expected_value_a'] > 0.05:
            status_a += "💎"
        
        # Fixed format specifier - separate percentage formatting from alignment
        model_a_str = f"{fight['model_prediction_a']:.1%}".ljust(8)
        market_a_str = f"{fight['market_prob_a']:.1%}".ljust(8)
        ev_a_str = f"{fight['expected_value_a']:+.1%}".ljust(8)
        
        print(f"{fight['fighter_a']:<20} {model_a_str} {market_a_str} {ev_a_str} {status_a}")
        
        # Fighter B - CORRECTED F-STRING FORMATTING  
        status_b = ""
        if fight['model_favorite'] == fight['fighter_b']:
            status_b += "⭐"
        if fight['expected_value_b'] > 0.05:
            status_b += "💎"
        
        # Fixed format specifier - separate percentage formatting from alignment
        model_b_str = f"{fight['model_prediction_b']:.1%}".ljust(8)
        market_b_str = f"{fight['market_prob_b']:.1%}".ljust(8)
        ev_b_str = f"{fight['expected_value_b']:+.1%}".ljust(8)
        
        print(f"{fight['fighter_b']:<20} {model_b_str} {market_b_str} {ev_b_str} {status_b}")
        
        # Additional info
        if fight['is_upset_opportunity']:
            print(f"   🚨 UPSET OPPORTUNITY - Model favors {fight['model_favorite']}, Market favors {fight['market_favorite']}")
        if fight['is_high_confidence']:
            print(f"   ⭐ HIGH CONFIDENCE - Strong model agreement with market")

# Generate predictions
try:
    print("🎯 Starting prediction analysis...")
    predictions = generate_fight_predictions()
    
    # Display detailed comparison
    display_fight_comparison_table(predictions)
    
    print(f"\\n✅ PREDICTIONS COMPLETE!")
    print(f"📊 Ready for bankroll analysis (Cell 4)!")
    
except Exception as e:
    print(f"❌ PREDICTION ANALYSIS FAILED")
    print(f"Error: {str(e)}")
    print(f"\\n💡 Ensure Cells 1 and 2 completed successfully")

🎯 Starting prediction analysis...
🧠 PREDICTION ENGINE WITH MARKET ANALYSIS
📅 Event: UFC_Fight_Night_Albazi_vs_Taira
🎯 Generating predictions for 2 fights...
\n🥊 Karol Rosa vs Nora Cornolle
   Analyzing: Karol Rosa vs Nora Cornolle
   🏆 Model Favorite: Karol Rosa (51.8%)
   📊 Market Favorite: Karol Rosa (63.7%)
   ⚔️  Method: KO/TKO
   🎯 Confidence: 51.8%
   💎 Best EV: Nora Cornolle (+26.7%)
\n🥊 Mateusz Rebecki vs Chris Duncan
   Analyzing: Mateusz Rebecki vs Chris Duncan
   🏆 Model Favorite: Mateusz Rebecki (74.0%)
   📊 Market Favorite: Mateusz Rebecki (66.7%)
   ⚔️  Method: Submission
   🎯 Confidence: 74.0%
   ⭐ HIGH CONFIDENCE PICK!
   💎 Best EV: Mateusz Rebecki (+11.0%)
\n📈 PREDICTION ANALYSIS SUMMARY
Total Fights: 2
Successful Predictions: 2
Failed Predictions: 0
Upset Opportunities: 0
High Confidence Picks: 1
\n🥊 Method Predictions:
   KO/TKO: 1 fights (50.0%)
   Submission: 1 fights (50.0%)
\n📊 DETAILED FIGHT ANALYSIS
\n🥊 Karol Rosa vs Nora Cornolle
Fighter              Model    

In [4]:
# 💰 CELL 4: DYNAMIC BANKROLL MANAGEMENT  
# ========================================
# Research-backed Kelly criterion with bankroll tier optimization

def determine_bankroll_strategy(bankroll: float) -> dict:
    """
    Determine optimal betting strategy based on bankroll size
    Research-backed approach from academic literature
    """
    if bankroll < 200:
        return {
            'tier': 'MICRO',
            'description': 'Ultra-conservative for small bankrolls',
            'kelly_multiplier': 0.15,  # Heavily fractional Kelly
            'max_single_bet_pct': 0.02,  # 2% max per bet
            'max_total_exposure': 0.10,  # 10% total exposure
            'min_ev_threshold': 0.12,  # Higher EV required (12%)
            'flat_bet_amount': max(5, bankroll * 0.01),  # $5 minimum or 1%
            'use_flat_betting': bankroll < 100,  # Use flat betting for very small amounts
            'strategy': 'Flat betting or 1/8 Kelly to preserve capital'
        }
    elif bankroll < 1000:
        return {
            'tier': 'SMALL',
            'description': 'Conservative growth strategy',
            'kelly_multiplier': 0.25,  # Quarter Kelly
            'max_single_bet_pct': 0.05,  # 5% max per bet
            'max_total_exposure': 0.20,  # 20% total exposure
            'min_ev_threshold': 0.08,  # Standard EV threshold
            'use_flat_betting': False,
            'strategy': '1/4 Kelly with strict risk management'
        }
    else:
        return {
            'tier': 'STANDARD',
            'description': 'Moderate Kelly with professional caps',
            'kelly_multiplier': 0.50,  # Half Kelly
            'max_single_bet_pct': 0.075,  # 7.5% max per bet
            'max_total_exposure': 0.25,  # 25% total exposure
            'min_ev_threshold': 0.05,  # Lower EV acceptable
            'use_flat_betting': False,
            'strategy': '1/2 Kelly with professional risk management'
        }

def calculate_optimal_bet_size(model_prob: float, decimal_odds: float, 
                              bankroll: float, strategy: dict, 
                              confidence_score: float = 1.0) -> dict:
    """
    Calculate optimal bet size using dynamic Kelly approach
    """
    # Calculate expected value
    ev = (model_prob * decimal_odds) - 1
    
    # Check minimum EV threshold
    if ev < strategy['min_ev_threshold']:
        return {
            'recommended_bet': 0,
            'reason': f"EV too low ({ev:.1%} < {strategy['min_ev_threshold']:.1%})",
            'ev': ev
        }
    
    # Calculate Kelly fraction
    kelly_fraction = ((model_prob * decimal_odds) - 1) / (decimal_odds - 1)
    
    if kelly_fraction <= 0:
        return {
            'recommended_bet': 0,
            'reason': "Negative Kelly fraction",
            'ev': ev
        }
    
    # Apply strategy-specific adjustments
    if strategy['use_flat_betting']:
        # Use flat betting for micro bankrolls
        bet_size = strategy['flat_bet_amount']
        sizing_method = "Flat betting (capital preservation)"
    else:
        # Use fractional Kelly
        adjusted_kelly = kelly_fraction * strategy['kelly_multiplier']
        bet_size = bankroll * adjusted_kelly
        sizing_method = f"{strategy['kelly_multiplier']:.0%} Kelly"
    
    # Apply confidence adjustment
    confidence_multiplier = 0.5 + (confidence_score * 0.5)  # 0.5 to 1.0 range
    bet_size *= confidence_multiplier
    
    # Apply maximum bet size cap
    max_bet = bankroll * strategy['max_single_bet_pct']
    if bet_size > max_bet:
        bet_size = max_bet
        sizing_method += " (capped)"
    
    # Apply minimum bet size
    min_bet = max(5, bankroll * 0.005)  # $5 minimum or 0.5%
    if bet_size < min_bet:
        if ev > strategy['min_ev_threshold'] * 1.5:  # Only bet if EV is significantly higher
            bet_size = min_bet
            sizing_method += " (minimum)"
        else:
            return {
                'recommended_bet': 0,
                'reason': f"Bet size too small (${bet_size:.2f} < ${min_bet:.2f})",
                'ev': ev
            }
    
    return {
        'recommended_bet': bet_size,
        'kelly_fraction': kelly_fraction,
        'adjusted_kelly': kelly_fraction * strategy['kelly_multiplier'],
        'sizing_method': sizing_method,
        'confidence_adjustment': confidence_multiplier,
        'ev': ev,
        'potential_profit': bet_size * (decimal_odds - 1),
        'roi_potential': ((bet_size * decimal_odds) - bet_size) / bet_size
    }

def generate_betting_recommendations():
    """
    Generate complete betting recommendations with dynamic bankroll management
    """
    print("💰 DYNAMIC BANKROLL MANAGEMENT")
    print("=" * 50)
    
    if not betting_system:
        raise Exception("❌ Betting system not initialized.")
    
    # FIXED: Use direct variable access with try/except pattern
    try:
        if not predictions or not predictions.get('fight_predictions'):
            raise Exception("❌ No predictions data available. Run Cell 3 first.")
    except NameError:
        raise Exception("❌ No predictions available. Run Cell 3 first.")
    
    current_bankroll = betting_system['bankroll']
    strategy = determine_bankroll_strategy(current_bankroll)
    
    print(f"💳 Bankroll Analysis:")
    print(f"   Amount: ${current_bankroll:,.2f}")
    print(f"   Tier: {strategy['tier']}")
    print(f"   Strategy: {strategy['strategy']}")
    print(f"   Max Single Bet: {strategy['max_single_bet_pct']:.1%}")
    print(f"   Min EV Required: {strategy['min_ev_threshold']:.1%}")
    
    recommendations = {
        'bankroll_info': {
            'amount': current_bankroll,
            'tier': strategy['tier'],
            'strategy': strategy
        },
        'single_bets': [],
        'rejected_bets': [],
        'portfolio_summary': {
            'total_recommended_stake': 0,
            'total_potential_profit': 0,
            'number_of_bets': 0,
            'portfolio_ev': 0,
            'bankroll_utilization': 0
        }
    }
    
    total_stake = 0
    total_potential_profit = 0
    
    print(f"\\n🎯 ANALYZING BETTING OPPORTUNITIES")
    print("=" * 45)
    
    for fight in predictions['fight_predictions']:
        fighter_a = fight['fighter_a']
        fighter_b = fight['fighter_b']
        
        print(f"\\n🥊 {fight['fight_key']}")
        
        # Analyze both fighters for betting opportunities
        fighters_to_analyze = [
            (fighter_a, fight['model_prediction_a'], fight['market_odds_a'], fight['expected_value_a']),
            (fighter_b, fight['model_prediction_b'], fight['market_odds_b'], fight['expected_value_b'])
        ]
        
        for fighter, model_prob, odds, ev in fighters_to_analyze:
            opponent = fighter_b if fighter == fighter_a else fighter_a
            
            # Check if we have enough exposure left
            remaining_exposure = (current_bankroll * strategy['max_total_exposure']) - total_stake
            if remaining_exposure < current_bankroll * 0.01:  # Less than 1% remaining
                recommendations['rejected_bets'].append({
                    'fighter': fighter,
                    'reason': 'Portfolio exposure limit reached',
                    'ev': ev
                })
                continue
            
            # Calculate optimal bet size
            bet_analysis = calculate_optimal_bet_size(
                model_prob, odds, current_bankroll, strategy, fight['confidence_score']
            )
            
            if bet_analysis['recommended_bet'] == 0:
                recommendations['rejected_bets'].append({
                    'fighter': fighter,
                    'reason': bet_analysis['reason'],
                    'ev': ev
                })
                continue
            
            # Adjust bet size if it would exceed remaining exposure
            final_bet_size = min(bet_analysis['recommended_bet'], remaining_exposure)
            
            # Create betting recommendation
            bet_recommendation = {
                'fighter': fighter,
                'opponent': opponent,
                'fight': fight['fight_key'],
                'model_probability': model_prob,
                'decimal_odds': odds,
                'american_odds': decimal_to_american(odds),
                'expected_value': ev,
                'kelly_fraction': bet_analysis['kelly_fraction'],
                'recommended_stake': final_bet_size,
                'potential_profit': final_bet_size * (odds - 1),
                'sizing_method': bet_analysis['sizing_method'],
                'confidence_score': fight['confidence_score'],
                'is_upset_opportunity': fight['is_upset_opportunity'],
                'is_high_confidence': fight['is_high_confidence'],
                'market_favorite': fight['market_favorite'],
                'model_favorite': fight['model_favorite']
            }
            
            recommendations['single_bets'].append(bet_recommendation)
            total_stake += final_bet_size
            total_potential_profit += bet_recommendation['potential_profit']
            
            print(f"   ✅ BET RECOMMENDATION: {fighter}")
            print(f"      Stake: ${final_bet_size:.2f} ({bet_analysis['sizing_method']})")
            print(f"      Odds: {odds:.2f} | EV: {ev:+.1%}")
            print(f"      Potential Profit: ${bet_recommendation['potential_profit']:.2f}")
            
            if fight['is_upset_opportunity']:
                print(f"      🚨 UPSET OPPORTUNITY!")
            if fight['is_high_confidence']:
                print(f"      ⭐ HIGH CONFIDENCE!")
    
    # Calculate portfolio metrics
    if recommendations['single_bets']:
        total_ev = sum(bet['expected_value'] * bet['recommended_stake'] for bet in recommendations['single_bets'])
        portfolio_ev = total_ev / total_stake if total_stake > 0 else 0
        
        recommendations['portfolio_summary'] = {
            'total_recommended_stake': total_stake,
            'total_potential_profit': total_potential_profit,
            'number_of_bets': len(recommendations['single_bets']),
            'portfolio_ev': portfolio_ev,
            'bankroll_utilization': total_stake / current_bankroll,
            'expected_return': total_ev
        }
    
    # Display summary
    print(f"\\n📊 BETTING RECOMMENDATIONS SUMMARY")
    print("=" * 45)
    
    if recommendations['single_bets']:
        summary = recommendations['portfolio_summary']
        
        # Sort recommendations by EV
        sorted_bets = sorted(recommendations['single_bets'], 
                           key=lambda x: x['expected_value'], reverse=True)
        
        print(f"🎯 RECOMMENDED BETS: {summary['number_of_bets']}")
        print(f"💰 Total Stakes: ${summary['total_recommended_stake']:.2f}")
        print(f"📈 Expected Return: ${summary['expected_return']:.2f}")
        print(f"🎲 Portfolio EV: {summary['portfolio_ev']:+.1%}")
        print(f"💳 Bankroll Utilization: {summary['bankroll_utilization']:.1%}")
        
        print(f"\\n🏆 TOP BETTING OPPORTUNITIES:")
        for i, bet in enumerate(sorted_bets[:5], 1):  # Show top 5
            print(f"{i}. {bet['fighter']} @ {bet['decimal_odds']:.2f}")
            print(f"   💰 Stake: ${bet['recommended_stake']:.2f}")
            print(f"   📈 EV: {bet['expected_value']:+.1%}")
            print(f"   🎯 Potential: ${bet['potential_profit']:.2f}")
            
            if bet['is_upset_opportunity']:
                print(f"   🚨 UPSET OPPORTUNITY")
            if bet['is_high_confidence']:
                print(f"   ⭐ HIGH CONFIDENCE")
            print()
        
        # Risk warning for high utilization
        if summary['bankroll_utilization'] > 0.20:
            print(f"⚠️  WARNING: High bankroll utilization ({summary['bankroll_utilization']:.1%})")
            print(f"   Consider reducing position sizes for first-time strategy")
    
    else:
        print(f"📭 NO BETTING OPPORTUNITIES FOUND")
        print(f"💡 Reasons:")
        rejection_reasons = {}
        for rejected in recommendations['rejected_bets']:
            reason = rejected['reason']
            if reason not in rejection_reasons:
                rejection_reasons[reason] = 0
            rejection_reasons[reason] += 1
        
        for reason, count in rejection_reasons.items():
            print(f"   • {reason}: {count} bets")
    
    return recommendations

# Execute bankroll analysis
try:
    print("💰 Starting dynamic bankroll analysis...")
    betting_recommendations = generate_betting_recommendations()
    
    print(f"\\n✅ BANKROLL ANALYSIS COMPLETE!")
    print(f"📊 Ready for bet tracking (Cell 5)!")
    
except Exception as e:
    print(f"❌ BANKROLL ANALYSIS FAILED")
    print(f"Error: {str(e)}")
    print(f"💡 Ensure previous cells completed successfully")

💰 Starting dynamic bankroll analysis...
💰 DYNAMIC BANKROLL MANAGEMENT
💳 Bankroll Analysis:
   Amount: $21.48
   Tier: MICRO
   Strategy: Flat betting or 1/8 Kelly to preserve capital
   Max Single Bet: 2.0%
   Min EV Required: 12.0%
\n🎯 ANALYZING BETTING OPPORTUNITIES
\n🥊 Karol Rosa vs Nora Cornolle
   ✅ BET RECOMMENDATION: Nora Cornolle
      Stake: $2.15 (Flat betting (capital preservation) (capped) (minimum))
      Odds: 2.63 | EV: +26.7%
      Potential Profit: $3.50
\n🥊 Mateusz Rebecki vs Chris Duncan
\n📊 BETTING RECOMMENDATIONS SUMMARY
🎯 RECOMMENDED BETS: 1
💰 Total Stakes: $2.15
📈 Expected Return: $0.57
🎲 Portfolio EV: +26.7%
💳 Bankroll Utilization: 10.0%
\n🏆 TOP BETTING OPPORTUNITIES:
1. Nora Cornolle @ 2.63
   💰 Stake: $2.15
   📈 EV: +26.7%
   🎯 Potential: $3.50

\n✅ BANKROLL ANALYSIS COMPLETE!
📊 Ready for bet tracking (Cell 5)!


In [5]:
# 📝 CELL 5: BET TRACKING AND CSV INTEGRATION
# ===========================================
# Professional bet logging with comprehensive tracking

def create_backup_and_validate_csv():
    """Create backup of betting records and validate structure"""
    csv_path = Path('/Users/diyagamah/Documents/ufc-predictor/betting_records.csv')
    
    # Create backup
    if csv_path.exists():
        backup_dir = Path('betting_backups')
        backup_dir.mkdir(exist_ok=True)
        
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        backup_path = backup_dir / f'betting_records_backup_{timestamp}.csv'
        
        shutil.copy2(csv_path, backup_path)
        print(f"🔒 Backup created: {backup_path}")
        
        # Validate existing structure with error handling
        try:
            df_existing = pd.read_csv(csv_path)
            print(f"📊 Current betting records: {len(df_existing)} bets")
            return True, df_existing
        except pd.errors.ParserError as e:
            print(f"⚠️  CSV parsing error: {str(e)}")
            print(f"💡 CSV may have inconsistent column structure")
            # Try to read with error handling
            try:
                df_existing = pd.read_csv(csv_path, error_bad_lines=False, warn_bad_lines=True)
                print(f"📊 Recovered {len(df_existing)} valid betting records")
                return True, df_existing
            except Exception as e2:
                print(f"❌ Could not recover CSV: {str(e2)}")
                return False, None
    else:
        print(f"📁 Creating new betting records file")
        return False, None

def generate_bet_records(recommendations: dict, event_name: str) -> List[dict]:
    """
    Generate bet records in the standard 24-column format
    """
    bet_records = []
    
    for i, bet in enumerate(recommendations['single_bets'], 1):
        # Generate unique bet ID
        bet_id = f"BET_{datetime.now().strftime('%Y%m%d_%H%M%S')}_{uuid.uuid4().hex[:8]}"
        
        # Determine risk level
        if bet['expected_value'] > 0.15 and bet['confidence_score'] > 0.8:
            risk_level = 'LOW'
        elif bet['expected_value'] > 0.08 and bet['confidence_score'] > 0.6:
            risk_level = 'MEDIUM'
        else:
            risk_level = 'HIGH'
        
        # Create comprehensive notes
        notes = []
        if bet['is_upset_opportunity']:
            notes.append("UPSET OPPORTUNITY")
        if bet['is_high_confidence']:
            notes.append("HIGH CONFIDENCE")
        notes.append(f"Model: {bet['model_probability']:.1%}")
        notes.append(f"Market: {(1/bet['decimal_odds']):.1%}")
        notes.append(f"Method: {bet['sizing_method']}")
        
        # Standard 24-column format - EXACTLY matching header
        bet_record = {
            'bet_id': bet_id,
            'date_placed': datetime.now().strftime('%Y-%m-%d'),
            'timestamp': datetime.now().isoformat(),
            'event': event_name,
            'fighter': bet['fighter'],
            'opponent': bet['opponent'],
            'bet_type': 'SINGLE',
            'odds_decimal': bet['decimal_odds'],
            'odds_american': bet['american_odds'],
            'bet_size': bet['recommended_stake'],
            'expected_value': bet['expected_value'],
            'model_probability': bet['model_probability'],
            'market_probability': 1 / bet['decimal_odds'],
            'risk_level': risk_level,
            'bankroll_at_time': recommendations['bankroll_info']['amount'],
            'bankroll_percentage': bet['recommended_stake'] / recommendations['bankroll_info']['amount'],
            'source': f"StreamlinedSystem_{recommendations['bankroll_info']['tier']}",
            'notes': " | ".join(notes),
            'actual_result': '',  # To be filled after fight
            'profit_loss': '',    # To be filled after fight
            'roi': '',           # To be filled after fight
            'result_updated': '', # To be filled after fight
            'fight_date': '',    # To be filled when known
            'method_actual': ''  # To be filled after fight
        }
        
        bet_records.append(bet_record)
        
        print(f"📝 Generated bet record {i}: {bet['fighter']} @ {bet['decimal_odds']:.2f}")
        print(f"   💰 ${bet['recommended_stake']:.2f} | EV: {bet['expected_value']:+.1%} | Risk: {risk_level}")
    
    return bet_records

def append_bets_to_csv(bet_records: List[dict]) -> bool:
    """
    Append bet records to the main CSV file with enhanced error handling
    """
    try:
        csv_path = Path('/Users/diyagamah/Documents/ufc-predictor/betting_records.csv')
        
        # Convert to DataFrame
        df_new_bets = pd.DataFrame(bet_records)
        
        # Ensure exact column order matches existing CSV
        expected_columns = [
            'bet_id', 'date_placed', 'timestamp', 'event', 'fighter', 'opponent', 
            'bet_type', 'odds_decimal', 'odds_american', 'bet_size', 'expected_value', 
            'model_probability', 'market_probability', 'risk_level', 'bankroll_at_time', 
            'bankroll_percentage', 'source', 'notes', 'actual_result', 'profit_loss', 
            'roi', 'result_updated', 'fight_date', 'method_actual'
        ]
        
        # Reorder columns to match exactly
        df_new_bets = df_new_bets[expected_columns]
        
        # Append to existing file or create new one
        if csv_path.exists():
            # Verify existing CSV structure before appending
            try:
                df_existing = pd.read_csv(csv_path)
                if len(df_existing.columns) != len(expected_columns):
                    print(f"⚠️  Column count mismatch: Expected {len(expected_columns)}, found {len(df_existing.columns)}")
                    print(f"💡 Creating new properly formatted CSV")
                    df_new_bets.to_csv(csv_path, index=False)
                    print(f"✅ Created new CSV with {len(bet_records)} bets")
                else:
                    df_new_bets.to_csv(csv_path, mode='a', header=False, index=False)
                    print(f"✅ Appended {len(bet_records)} bets to existing CSV")
            except Exception as e:
                print(f"⚠️  Error reading existing CSV: {str(e)}")
                print(f"💡 Creating new CSV file")
                df_new_bets.to_csv(csv_path, index=False)
                print(f"✅ Created new CSV with {len(bet_records)} bets")
        else:
            df_new_bets.to_csv(csv_path, index=False)
            print(f"✅ Created new CSV with {len(bet_records)} bets")
        
        # Verify the operation worked
        try:
            df_verification = pd.read_csv(csv_path)
            print(f"📊 Total bets in CSV: {len(df_verification)}")
        except Exception as e:
            print(f"⚠️  Could not verify CSV: {str(e)}")
        
        return True
        
    except Exception as e:
        print(f"❌ Failed to append bets to CSV: {str(e)}")
        return False

def export_analysis_summary(recommendations: dict, predictions: dict, event_name: str):
    """
    Export comprehensive analysis summary
    """
    try:
        # Create analysis export
        analysis_dir = Path('analysis_exports')
        analysis_dir.mkdir(exist_ok=True)
        
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        export_path = analysis_dir / f'analysis_{event_name}_{timestamp}.json'
        
        # Compile comprehensive analysis data
        analysis_export = {
            'metadata': {
                'event_name': event_name,
                'analysis_timestamp': datetime.now().isoformat(),
                'bankroll_tier': recommendations['bankroll_info']['tier'],
                'bankroll_amount': recommendations['bankroll_info']['amount']
            },
            'predictions_summary': predictions['summary'],
            'betting_summary': recommendations['portfolio_summary'],
            'recommended_bets': recommendations['single_bets'],
            'rejected_opportunities': recommendations['rejected_bets'],
            'fight_analysis': predictions['fight_predictions']
        }
        
        # Save analysis
        with open(export_path, 'w') as f:
            json.dump(analysis_export, f, indent=2, default=str)
        
        print(f"📈 Analysis exported: {export_path}")
        
        return export_path
        
    except Exception as e:
        print(f"⚠️  Analysis export failed: {str(e)}")
        return None

def display_final_betting_sheet(recommendations: dict):
    """
    Display professional betting sheet format
    """
    print(f"\n📋 FINAL BETTING SHEET")
    print("=" * 80)
    
    if not recommendations['single_bets']:
        print(f"📭 No bets recommended for this event")
        return
    
    # Header
    print(f"Event: {odds_result['event_name']}")
    print(f"Bankroll: ${recommendations['bankroll_info']['amount']:,.2f} ({recommendations['bankroll_info']['tier']} tier)")
    print(f"Strategy: {recommendations['bankroll_info']['strategy']['strategy']}")
    print(f"Total Recommended: ${recommendations['portfolio_summary']['total_recommended_stake']:.2f}")
    print(f"Expected Return: ${recommendations['portfolio_summary']['expected_return']:.2f}")
    print(f"Portfolio EV: {recommendations['portfolio_summary']['portfolio_ev']:+.1%}")
    
    print(f"\n{'#':<3} {'Fighter':<25} {'Odds':<8} {'Stake':<10} {'EV':<8} {'Profit':<10} {'Type'}")
    print("-" * 80)
    
    # Sort by EV for display
    sorted_bets = sorted(recommendations['single_bets'], 
                        key=lambda x: x['expected_value'], reverse=True)
    
    for i, bet in enumerate(sorted_bets, 1):
        bet_type = ""
        if bet['is_upset_opportunity']:
            bet_type += "🚨"
        if bet['is_high_confidence']:
            bet_type += "⭐"
        if not bet_type:
            bet_type = "📊"
        
        print(f"{i:<3} {bet['fighter']:<25} {bet['decimal_odds']:<8.2f} "
              f"${bet['recommended_stake']:<9.2f} {bet['expected_value']:<7.1%} "
              f"${bet['potential_profit']:<9.2f} {bet_type}")
    
    print("-" * 80)
    print(f"{'TOTAL':<39} ${recommendations['portfolio_summary']['total_recommended_stake']:<9.2f} "
          f"{recommendations['portfolio_summary']['portfolio_ev']:<7.1%} "
          f"${recommendations['portfolio_summary']['total_potential_profit']:<9.2f}")

def execute_bet_tracking():
    """
    Execute complete bet tracking workflow
    """
    print("📝 COMPREHENSIVE BET TRACKING SYSTEM")
    print("=" * 50)
    
    # Validate prerequisites
    if not betting_system:
        raise Exception("❌ Betting system not initialized.")
    
    # Check for betting_recommendations variable
    try:
        betting_recs = betting_recommendations
        if not betting_recs or not betting_recs.get('single_bets'):
            raise Exception("❌ No betting recommendations available. Run Cell 4 first.")
    except NameError:
        raise Exception("❌ No betting recommendations available. Run Cell 4 first.")
    
    # Check for predictions variable 
    try:
        preds = predictions
        if not preds or not preds.get('fight_predictions'):
            raise Exception("❌ No predictions available. Run Cell 3 first.")
    except NameError:
        raise Exception("❌ No predictions available. Run Cell 3 first.")
    
    # Check if we have bets to track
    if not betting_recs['single_bets']:
        print(f"📭 No bets to track - no profitable opportunities found")
        print(f"\n✅ ANALYSIS COMPLETE!")
        return
    
    event_name = odds_result['event_name']
    
    print(f"📅 Event: {event_name}")
    print(f"🎯 Processing {len(betting_recs['single_bets'])} recommended bets")
    
    # Step 1: Create backup and validate
    print(f"\n🔒 Creating backup and validating CSV...")
    file_exists, existing_df = create_backup_and_validate_csv()
    
    # Step 2: Generate bet records
    print(f"\n📝 Generating bet records...")
    bet_records = generate_bet_records(betting_recs, event_name)
    
    # Step 3: Append to CSV
    print(f"\n💾 Appending to betting records CSV...")
    append_success = append_bets_to_csv(bet_records)
    
    if not append_success:
        print(f"❌ Failed to save bets - manual backup required")
        return
    
    # Step 4: Export analysis summary
    print(f"\n📈 Exporting analysis summary...")
    export_path = export_analysis_summary(betting_recs, preds, event_name)
    
    # Step 5: Display final betting sheet
    display_final_betting_sheet(betting_recs)
    
    # Step 6: Final summary
    print(f"\n🎉 BET TRACKING COMPLETE!")
    print(f"✅ Logged {len(bet_records)} bets to CSV")
    print(f"💾 CSV Path: /Users/diyagamah/Documents/ufc-predictor/betting_records.csv")
    if export_path:
        print(f"📈 Analysis: {export_path}")
    
    print(f"\n📊 SESSION SUMMARY:")
    print(f"   Event: {event_name}")
    print(f"   Bets Placed: {len(bet_records)}")
    print(f"   Total Stakes: ${betting_recs['portfolio_summary']['total_recommended_stake']:.2f}")
    print(f"   Expected Return: ${betting_recs['portfolio_summary']['expected_return']:.2f}")
    print(f"   Bankroll Utilization: {betting_recs['portfolio_summary']['bankroll_utilization']:.1%}")
    
    print(f"\n🎯 READY FOR BETTING!")
    print(f"💡 Remember to update results after fights for performance tracking")

# Execute the complete bet tracking workflow
try:
    execute_bet_tracking()
    
except Exception as e:
    print(f"❌ BET TRACKING FAILED")
    print(f"Error: {str(e)}")
    print(f"\n💡 Troubleshooting:")
    print(f"   1. Ensure all previous cells completed successfully")
    print(f"   2. Check file permissions for CSV writing")
    print(f"   3. Verify betting_records.csv path exists")

📝 COMPREHENSIVE BET TRACKING SYSTEM
📅 Event: UFC_Fight_Night_Albazi_vs_Taira
🎯 Processing 1 recommended bets

🔒 Creating backup and validating CSV...
🔒 Backup created: betting_backups/betting_records_backup_20250801_121316.csv
📊 Current betting records: 8 bets

📝 Generating bet records...
📝 Generated bet record 1: Nora Cornolle @ 2.63
   💰 $2.15 | EV: +26.7% | Risk: HIGH

💾 Appending to betting records CSV...
✅ Appended 1 bets to existing CSV
📊 Total bets in CSV: 9

📈 Exporting analysis summary...
📈 Analysis exported: analysis_exports/analysis_UFC_Fight_Night_Albazi_vs_Taira_20250801_121316.json

📋 FINAL BETTING SHEET
Event: UFC_Fight_Night_Albazi_vs_Taira
Bankroll: $21.48 (MICRO tier)
Strategy: Flat betting or 1/8 Kelly to preserve capital
Total Recommended: $2.15
Expected Return: $0.57
Portfolio EV: +26.7%

#   Fighter                   Odds     Stake      EV       Profit     Type
--------------------------------------------------------------------------------
1   Nora Cornolle      